# 3. Train-Predict KNN
Result:
- Kaggle score:

Reference:
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
- http://scikit-learn.org/stable/modules/model_evaluation.html#model-evaluation

## Run name

In [19]:
import time
import os
import pandas as pd
import gc

project_name = 'Google_LandMark_Rec'
step_name = 'Train-Predict_KNN'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: Google_LandMark_Rec_Train-Predict_KNN_20180322_133359


## 项目文件夹

In [20]:
cwd = os.getcwd()
data1_folder = '/data1/kaggle/landmark-recognition-challenge/'
input_folder = os.path.join(data1_folder, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t' + input_folder)
print('output_folder: \t' + output_folder)
print('model_folder: \t' + model_folder)
print('feature_folder: \t' + feature_folder)
print('post_pca_feature_folder: \t' + post_pca_feature_folder)
print('log_folder: \t' + log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

input_folder: 	/data1/kaggle/landmark-recognition-challenge/input
output_folder: 	/data2/kaggle/landmark-recognition-challenge/output
model_folder: 	/data2/kaggle/landmark-recognition-challenge/model
feature_folder: 	/data2/kaggle/landmark-recognition-challenge/feature
post_pca_feature_folder: 	/data2/kaggle/landmark-recognition-challenge/post_pca_feature
log_folder: 	/data2/kaggle/landmark-recognition-challenge/log


In [21]:
train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## 加载feature

In [22]:
%%time
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2018)

x_data = []
y_data = {}
x_test = []


image_size = 150
time_str = '20180311-151108'
cwd = os.getcwd()
# feature_cgg16 = os.path.join(cwd, 'feature', 'feature_VGG16_{}.h5'.format(20180219))
# feature_cgg19 = os.path.join(cwd, 'feature', 'feature_VGG19_{}.h5'.format(20180219))
# feature_resnet50 = os.path.join(cwd, 'feature', 'feature_ResNet50_{}.h5'.format(20180220))
# feature_xception = os.path.join(cwd, 'feature', 'feature_Xception_{}.h5'.format(20180221))
feature_inceptionV3 = os.path.join(cwd, 'post_pca_feature', 'post_pca_feature_InceptionV3_%s_%s.h5' % (image_size, time_str))
# feature_inceptionResNetV2 = os.path.join(cwd, 'feature', 'feature_InceptionResNetV2_%s.h5' % time_str)
# for filename in [feature_cgg16, feature_cgg19, feature_resnet50, feature_xception, feature_inception, feature_inceptionResNetV2]:
for filename in [feature_inceptionV3]:
    with h5py.File(filename, 'r') as h:
        x_data.append(np.array(h['train']))
        y_data = np.array(h['train_labels'])
#         x_val.append(np.array(h['val']))
#         y_val = np.array(h['val_labels'])
        x_test.append(np.array(h['test']))

CPU times: user 76 ms, sys: 356 ms, total: 432 ms
Wall time: 431 ms


In [23]:
print(x_data[0].shape)
print(len(y_data))
print(x_test[0].shape)

(1219426, 32)
1219426
(115942, 32)


In [24]:
%%time
x_data = np.concatenate(x_data, axis=-1)
x_test = np.concatenate(x_test, axis=-1)
print(x_data.shape)
print(x_test.shape)

(1219426, 32)
(115942, 32)
CPU times: user 92 ms, sys: 132 ms, total: 224 ms
Wall time: 221 ms


In [25]:
# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.0025, random_state=5)

# print(x_train.shape)
# print(x_val.shape)
# print(y_train.shape)
# print(y_val.shape)

## Train

In [26]:
set_y_data = list(set(y_data))
print(len(set_y_data))

14951


In [27]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2017)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_test.shape)


(1097483, 32)
(121943, 32)
(1097483,)
(121943,)
(115942, 32)


In [28]:
set_y_train = list(set(y_train))
print(len(set_y_train))

14935


In [29]:
%%time
from sklearn.neighbors import NearestNeighbors

clf = NearestNeighbors(
    radius=2,
    algorithm='brute',
    leaf_size=10,
    metric='cosine',
    n_jobs=-1
)
clf.fit(x_train, y_train)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 23.3 ms


In [59]:
%%time
from sklearn.metrics import accuracy_score
y_train_pred = clf.kneighbors(x_train[:100], 20, True)
print(y_train_pred[0].shape)
print(y_train_pred[1].shape)

(100, 20)
(100, 20)
CPU times: user 2.56 s, sys: 2.9 s, total: 5.46 s
Wall time: 6.35 s


[0.0, 0.06538951683659111, 0.06976698678855286, 0.06549858852544566, 0.07565495677697354, 0.07852375768471087, 0.07917262147236381, 0.08025981783868485, 0.08280597356697061, 0.08461551578116666, 0.0881239217552593, 0.08893575436149581, 0.0895788925283868, 0.09348210533366663, 0.09239025829460157, 0.08668538853937968, 0.08760842962674009, 0.08951283647733077, 0.09247505259113598, 0.09182362162577673]
[684352, 1, 231906, 168261, 823398, 762919, 621341, 383481, 921486, 867992, 888368, 869745, 11439, 347317, 274230, 769495, 557277, 801529, 682525, 1036511]


In [31]:
y_train_pred1 = [y_train[i] for i in y_train_pred[1][:, 0]]
# print(y_train_pred1.shape)
print(y_train_pred1[:10])
print(accuracy_score(y_train[:100], y_train_pred1))

[3063, 2257, 2241, 6716, 8776, 12172, 585, 12965, 6669, 12988]
1.0


In [69]:
%%time
from sklearn.metrics import accuracy_score
count = 100
# count = len(y_val)

y_val_nbs = clf.kneighbors(x_val[:count], 10, True)
print(y_val_nbs[0].shape)
print(y_val_nbs[1].shape)

(100, 10)
(100, 10)
CPU times: user 2.64 s, sys: 2.9 s, total: 5.54 s
Wall time: 6.46 s


In [71]:
%%time
y_val_pred = [y_train[i] for i in y_val_nbs[1][:, 0]]

# for i in range(count):
#     print((y_val[i], y_val_pred1[i]), end=' ')
#     if y_val[i] == y_val_pred1[i]:
#         print('*')
#     else:
#         print(' ')

val_acc = accuracy_score(y_val[:count], y_val_pred)
print(val_acc)

0.29
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 553 µs


In [72]:
print(list(set(y_val_nbs[0][1])))
print(list(set(y_val_nbs[1][1])))

[0.024222510077095305, 0.030797483111828616, 0.03797095245108939, 0.04052729766744845, 0.03930628382825674, 0.040626142672073096, 0.040736926042068244, 0.04152736515826572, 0.04306521695107457, 0.04306807037333271]
[1027454, 393475, 859748, 991173, 675247, 129392, 938225, 767762, 448667, 76254]


In [38]:
%%time
y_test_nbs = clf.kneighbors(x_test, 1, True)
print(y_test_nbs[0].shape)
print(y_test_nbs[1].shape)

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.6/site-packages/ipykernel/__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home

In [17]:
%%time
y_test_pred = [y_train[i] for i in y_test_nbs[1][:, 0]]

NameError: name 'y_test_nbs' is not defined

In [18]:
print(len(y_test_pred))
print(y_test_pred[:10])

NameError: name 'y_test_pred' is not defined

## Predict

In [ ]:
sample_submission_csv = pd.read_csv(sample_submission_folder)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

In [ ]:
# 这里证明os.listdir()得到的图片名称list不正确
files = os.listdir(os.path.join(input_folder, 'data_test', 'test'))
print(files[:10])

In [47]:
# 这里证明ImageDataGenerator()得到的图片名称list才是正确
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator()
image_size = (299, 299)
batch_size = 128
test_generator  = gen.flow_from_directory(test_folder, image_size, shuffle=False, batch_size=batch_size)
print('test_generator')
print(len(test_generator.filenames))
print(test_generator.filenames[:10])

Using TensorFlow backend.


Found 115942 images belonging to 1 classes.
test_generator
115942
['test/000088da12d664db.jpg', 'test/0001623c6d808702.jpg', 'test/0001bbb682d45002.jpg', 'test/0002362830cfe3a3.jpg', 'test/000270c9100de789.jpg', 'test/0002b0fab5d3ccc4.jpg', 'test/000396be3c24830a.jpg', 'test/000506dc6ab3a40e.jpg', 'test/0005292fc4b005a3.jpg', 'test/0005456a82264bc8.jpg']


In [53]:
# test_generator_filenames = np.array(test_generator.filenames)
# test_generator_filenames_file = os.path.join(output_folder, 'test_generator_filenames.npy')

# np.save(test_generator_filenames_file, test_generator_filenames)

In [ ]:
%%time

test_dict = {}
for i, paire in enumerate(zip(test_generator.filenames, y_test_pred)):
    image_name, label = paire[0], paire[1]
    image_id = image_name[5:-4]
    test_dict[image_id] = '%d %.4f' % (label, 1)

#确认图片的id是否能与ImageDataGenerator()对应上
for key in list(test_dict.keys())[:10]:
    print('%s  %s' % (key, test_dict[key]))

In [ ]:
%%time
len_sample_submission_csv = len(sample_submission_csv)
print('len(len_sample_submission_csv)=%d' % len_sample_submission_csv)
count = 0
for i in range(len_sample_submission_csv):
    image_id = sample_submission_csv.iloc[i, 0]
#     landmarks = sample_submission_csv.iloc[i, 1]
    if image_id in test_dict:
        pred_landmarks = test_dict[image_id]
#         print('%s  %s' % (image_id, pred_landmarks))
        sample_submission_csv.iloc[i, 1] = pred_landmarks
    else:
#         print(image_id)
#         sample_submission_csv.iloc[i, 1] = '9633 1.0' # 属于9633的类最多，所以全都设置成这个类，可能会比设置成空得到的结果好
        sample_submission_csv.iloc[i, 1] = '' # 设置成空
    count += 1
    if count % 10000 == 0:
        print(int(count/10000), end=' ')
display(sample_submission_csv.head(2))

In [ ]:
run_name_acc = run_name + '_' + str(int(val_acc*10000)).zfill(4)
pred_file = os.path.join(output_folder, 'pred_' + run_name_acc + '.csv')
print(pred_file)

sample_submission_csv.to_csv(pred_file, index=None)

In [ ]:
print(run_name_acc)
print('Done !')